# Columbia University - IEOR 4732: Computational Methods in Finance
# Homework 2 by Alexandre Duhamel - afd2153
<br><br>

### Q1 - Use explicit-implicit finite difference scheme covered during the lecture to solve the PIDE.

In [27]:
import numpy as np
import scipy.linalg as la

In [28]:
# # Parameters
# S0 = 1900  # Spot price
# K = 2000  # Strike price
# B = 2200  # Barrier
# r = 0.0025  # Risk-free rate 
# q = 0.015  # Dividend rate
# T = 0.5  # Maturity
# sigma = 0.25  # Volatility
# nu = 0.31  # Jump intensity
# theta = -0.25  # Mean jump size
# Y = 0.4  # Tail index

# # Compute lambda parameters
# lambda_p = ((theta**2 / sigma**4) + (2 / (sigma**2 * nu)))**(1/2) - (theta / sigma**2)
# lambda_n = ((theta**2 / sigma**4) + (2 / (sigma**2 * nu)))**(1/2) + (theta / sigma**2)

# print("lambda_p =", lambda_p, "lambda_n =", lambda_n)

# # Grid parameters
# N = 100  # Number of spatial steps
# M = 100  # Number of time steps

# # Define the spatial grid range in log-space
# x_min = np.log(S0) - 1 # Lower bound in log-space
# x_max = np.log(B)  # Upper bound (log barrier level)

# # D = 
# # x_i = x_min + i * dx, i = 0, 1, ..., N
# # t_j = 0 + j * dt, j = 0, 1, ..., M
# dx = (x_max - x_min) / (N)  # Spatial step size
# dt = T / M  # Time step size

# # Discretized spatial and time grids
# x_grid = np.linspace(x_min, x_max, N)  # N+1 grid points in space
# tau_grid = np.linspace(0, T, M)  # M+1 grid points in time

# # Display the computed grid parameters
# grid_params = {
#     "x_min": x_min,
#     "x_max": x_max,
#     "dx": dx,
#     "dt": dt,
#     "Number of spatial points": N,
#     "Number of time points": M,
# }

# grid_params

In [29]:
# Compute k(y) Lévy-measure function
def k_y(y, lambda_n, lambda_p, nu, Y):
    result = []
    for val in y:
        if val < 0:
            output = np.exp(-lambda_n * np.abs(val)) / (nu * np.abs(val)**(1 + Y))
        elif val > 0:
            output = np.exp(-lambda_p * val) / (nu * val**(1 + Y))
        else:
            output = 0.0
        result.append(output)
    return np.array(result)

# y_values = np.array([-2, -1, -0.5, -0.1, 0.1, 0.5, 1, 2])
# k_values = k_y(y_values, lambda_n, lambda_p, nu, Y)

# print("y values:", y_values)
# print("k(y) values:", k_values)

$w(x,\tau)$ is a **matrix** where:

- **Rows** correspond to spatial points $x$ (log stock prices).
- **Columns** correspond to time points $\tau$ (backward in time from $T$ to $0$).

Since we discretize the problem using a grid of $N$ spatial steps and $M$ time steps, we initialize $w$ as an $N \times M$ matrix, where:

- The **first column** corresponds to the initial condition (the option payoff at $\tau = 0$).
- The **last row** corresponds to the barrier condition $w(B, \tau) = 0$.
- The **first row** is set to $w(x_0, \tau) = 0$ for all $\tau$ (absorbing boundary condition).

In [30]:
import scipy.integrate as spi

def g1(xi):
        return spi.quad(lambda z: np.exp(-z) / z**Y, xi, np.inf)[0]
    
def g2(xi):
    return spi.quad(lambda z: np.exp(-z) / z**(Y+1), xi, np.inf)[0]

In [31]:
# Function to compute B_l and B_u coefficients
def compute_B_coefficients(dx, dt, lambda_n, lambda_p, nu, Y, r, q, pre_computed=True):
    """ Compute B_l and B_u coefficients for the PIDE discretization """
    
    # Compute σ²(ε)
    def sigma_squared_epsilon(epsilon, pre_computed=True):
        """ Compute σ²(ε) = ∫ |y|≤ε y² k(y) dy """
        if pre_computed:
            term1 =  (-(lambda_p * epsilon)**(1 - Y)) * np.exp(-lambda_p * epsilon)
            term2 = (1 - Y) * (g1(0) - g1(lambda_p * epsilon))
            
            term3 =  (-(lambda_n * epsilon)**(1 - Y)) * np.exp(-lambda_n * epsilon)
            term4 = (1 - Y) * (g1(0) - g1(lambda_n * epsilon))

            sigma2_eps = (1) * lambda_p**(Y - 2) * (term1 + term2) + (1) * lambda_n**(Y - 2) * (term3 + term4)
            # print(f"sigma2_eps: {sigma2_eps}, term1: {term1}, term2: {term2}, term3: {term3}, term4: {term4}")
            return sigma2_eps
        else:
            y_vals = np.linspace(-epsilon, epsilon, 1000)  
            integral_values = y_vals**2 * k_y(y_vals, lambda_n, lambda_p, nu, Y)
            return np.trapz(integral_values, y_vals)


    # Compute ω(ε)
    def omega_epsilon(epsilon, pre_computed=True):
        """ Compute ω(ε) = ∫ |y|>ε (1 - e^y) k(y) dy """
        if pre_computed:
            term1 = (lambda_p**Y) * g2(lambda_p * epsilon)
            term2 = ((lambda_p - 1)**Y / nu) * g2((lambda_p - 1) * epsilon)
            term3 = (lambda_n**Y) * g2(lambda_n * epsilon)
            term4 = ((lambda_n + 1)**Y) * g2((lambda_n + 1) * epsilon)

            omega_eps = term1 - term2 + term3 - term4
            # print(f"omega_eps: {omega_eps}, term1: {term1}, term2: {term2}, term3: {term3}, term4: {term4}")
            return omega_eps
        else:
            y_vals = np.linspace(epsilon, 10 * epsilon, 1000)  # Approximate upper bound
            integral_values = (1 - np.exp(y_vals)) * k_y(y_vals, lambda_n, lambda_p, nu, Y)
            return np.trapz(integral_values, y_vals)

    # Compute σ²(ε) and ω(ε) once
    sigma2_eps = sigma_squared_epsilon(dx)
    omega_eps = omega_epsilon(dx)

    # Compute B_l and B_u
    B_l = (sigma2_eps * dt) / (2 * dx**2) - ((r - q + omega_eps - 0.5 * sigma2_eps) * (dt / (2 * dx)))
    B_u = (sigma2_eps * dt) / (2 * dx**2) + ((r - q + omega_eps - 0.5 * sigma2_eps) * (dt / (2 * dx)))

    return B_l, B_u

# B_l, B_u = compute_B_coefficients(dx, dt, lambda_n, lambda_p, nu, Y, r, q, pre_computed=True)
# print("Pre-computed :")
# print("B_l =", B_l)
# print("B_u =", B_u)

# B_l, B_u = compute_B_coefficients(dx, dt, lambda_n, lambda_p, nu, Y, r, q, pre_computed=False)
# print("Computed :")
# print("B_l =", B_l)
# print("B_u =", B_u)

In [32]:
import numpy as np
import scipy.integrate as integrate

# Compute the integral term FOR A CALL ∫_{|y| > Δx} (w(xi + y, τj) − wi,j)k(y)dy
# --> won't be used in the final implementation since we have a direct Rij computation... oh well
def compute_large_jump_integral(w, x_grid, K, N, i, j, dx, nu, lambda_p, lambda_n, Y, g1_n, g1_p, g2_n, g2_p, g2_n_shifted, g2_p_shifted):
    """ Compute the jump integral over |y| > Δx by breaking it into four sub-intervals """
    def integral_1():
        """ Integral over (-∞, x_0 - x_i) """
        return 0 #(lambda_n**Y / nu) * (- w[i, j]) * g2_n(i-1) # already included in diagonal componant
    
    def integral_2():
        """ Integral over (x_0 - x_i, -Δx) """
        return sum([
            (lambda_n**Y) * (w[i-k, j] - w[i, j] - k*(w[i-k-1,j] - w[i-k,j])) * (g2_n[k-1] - g2_n[k]) + 
            (w[i-k-1, j] - w[i-k, j]) / (nu * lambda_n**(1-Y) * dx) * (g1_n[k-1] - g1_n[k])
            for k in range(1, i)
        ])
    
    def integral_3():
        """ Integral over (+Δx, x_N - x_i) """
        return sum([
            (lambda_p**Y) * (w[i+k, j] - w[i, j] - k*(w[i+k+1,j] - w[i+k,j])) * (g2_p[k-1] - g2_p[k]) + 
            (w[i+k+1, j] - w[i+k, j]) / (nu * lambda_p**(1-Y) * dx) * (g1_p[k-1] - g1_p[k]) 
            for k in range(1, N-i-1)
        ])
    
    def integral_4():
        """ Integral over (x_N - x_i, ∞) """
        return ((1+lambda_p)**Y) * np.exp(x_grid[i]) * g2_p_shifted[N-i-1] - ((lambda_p**Y)) * K * g2_p[N-i-1]
    
    return integral_1() + integral_2() + integral_3() + integral_4()

In [33]:
# # Function to compute l, d, u coefficients and R term
# def compute_coefficients_put(w, x_grid, N, K, i, j, dx, dt, B_l, B_u, lambda_n, lambda_p, Y, r, nu, precomputed_g_values=None):
#     """ Compute coefficients l, d, u and R term for the PIDE discretization """
#     if precomputed_g_values:
#         # Extract precomputed g1 and g2 values
#         g1_n = precomputed_g_values["g1_n"]
#         g1_p = precomputed_g_values["g1_p"]
#         g2_n = precomputed_g_values["g2_n"]
#         g2_p = precomputed_g_values["g2_p"]
#         g2_n_shifted = precomputed_g_values["g2_n_shifted"]
#         g2_p_shifted = precomputed_g_values["g2_p_shifted"]
#     else:
#         # Compute g1 and g2 values dynamically
#         g1_n = np.array([g1(k * lambda_n * dx) for k in range(1, N)])
#         g1_p = np.array([g1(k * lambda_p * dx) for k in range(1, N)])
#         g2_n = np.array([g2(k * lambda_n * dx) for k in range(1, N)])
#         g2_p = np.array([g2(k * lambda_p * dx) for k in range(1, N)])
#         g2_n_shifted = np.array([g2(k * (lambda_n + 1) * dx) for k in range(1, N)])
#         g2_p_shifted = np.array([g2(k * (lambda_p - 1) * dx) for k in range(1, N)])

#     # Compute l, d, u coefficients
#     l_ij1 = -B_l
#     d_ij1 = 1 + r * dt + B_l + B_u + (dt / nu) * (
#         lambda_n**Y * g2_n[i-1] + lambda_p**Y * g2_p[N - i - 1]
#     )
#     u_ij1 = -B_u

#     # Compute R_i,j
#     R_ij = 0
#     for k in range(1, i):
#         R_ij += lambda_n**Y * (w[i-k, j] - w[i, j] - k * (w[i-k-1, j] - w[i-k, j])) * (g2_n[k-1] - g2_n[k])
#         R_ij += (w[i-k-1, j] - w[i-k, j]) / (lambda_n**(1-Y) * dx) * (g1_n[k-1] - g1_n[k])

#     for k in range(1, N-i-1): 
#         R_ij += lambda_p**Y * (w[i+k, j] - w[i, j] - k * (w[i+k+1, j] - w[i+k, j])) * (g2_p[k-1] - g2_p[k])
#         R_ij += (w[i+k+1, j] - w[i+k, j]) / (lambda_p**(1-Y) * dx) * (g1_p[k-1] - g1_p[k])

#     R_ij += K * lambda_n**Y * g2_n[i-1] - np.exp(x_grid[i]) * (lambda_n + 1)**Y * g2_n_shifted[i-1]

#     return l_ij1, d_ij1, u_ij1, R_ij

# # w = np.zeros((N, M))
# # w[:, 0] = np.maximum(np.exp(x_grid) - K, 0)
# # for j in range(1, M):
# #     w[:, j] = w[:, 0]

# # i = N // 2  # e.g., middle index
# # j = 0       # first time step

# # # Test using precomputed g-values
# # l_ij1_pre, d_ij1_pre, u_ij1_pre, R_ij_pre = compute_coefficients(
# #     w, x_grid, N, K, i, j, dx, dt, B_l, B_u, lambda_n, lambda_p, Y, r, nu, precomputed_g_values
# # )
# # print("Pre-computed:")
# # print("l_ij1, d_ij1, u_ij1, R_ij =", l_ij1_pre, d_ij1_pre, u_ij1_pre, R_ij_pre)

# # # Test using dynamic (computed) g-values
# # l_ij1_dyn, d_ij1_dyn, u_ij1_dyn, R_ij_dyn = compute_coefficients(
# #     w, x_grid, N, K, i, j, dx, dt, B_l, B_u, lambda_n, lambda_p, Y, r, nu, precomputed_g_values=None
# # )
# # print("Computed:")
# # print("l_ij1, d_ij1, u_ij1, R_ij =", l_ij1_dyn, d_ij1_dyn, u_ij1_dyn, R_ij_dyn)

In [34]:
# Function to compute l, d, u coefficients and R term
def compute_coefficients_call(w, x_grid, N, K, i, j, dx, dt, B_l, B_u, lambda_n, lambda_p, Y, r, nu, precomputed_g_values=None):
    """ Compute coefficients l, d, u and R term for the PIDE discretization """
    if precomputed_g_values is None:
        raise ValueError("Precomputed g1 and g2 values must be provided. Recomputing them is not allowed for efficiency reasons.")

    # Extract precomputed g1 and g2 values
    g1_n = precomputed_g_values.get("g1_n")
    g1_p = precomputed_g_values.get("g1_p")
    g2_n = precomputed_g_values.get("g2_n")
    g2_p = precomputed_g_values.get("g2_p")
    g2_n_shifted = precomputed_g_values.get("g2_n_shifted")
    g2_p_shifted = precomputed_g_values.get("g2_p_shifted")
    
    # Ensure all required keys exist in precomputed_g_values
    missing_keys = [key for key in ["g1_n", "g1_p", "g2_n", "g2_p", "g2_n_shifted", "g2_p_shifted"] if key not in precomputed_g_values]
    if missing_keys:
        raise ValueError(f"Missing precomputed g-values: {', '.join(missing_keys)}")

    # Compute l, d, u coefficients
    l_ij1 = -B_l
    d_ij1 = 1 + r * dt + B_l + B_u
    u_ij1 = -B_u

    # Compute R_i,j
    jump_integral = compute_large_jump_integral(w, x_grid, K, N, i, j, dx, nu, lambda_p, lambda_n, Y, g1_n, g1_p, g2_n, g2_p, g2_n_shifted, g2_p_shifted) - (lambda_n**Y * g2_n[i-1] + lambda_p**Y * g2_p[N - i - 1]) * w[i,j]
    R_ij = jump_integral
    
    return l_ij1, d_ij1, u_ij1, R_ij

# w = np.zeros((N, M))
# w[:, 0] = np.maximum(np.exp(x_grid) - K, 0)
# for j in range(1, M):
#     w[:, j] = w[:, 0]

# i = N // 2  # e.g., middle index
# j = 0       # first time step

# # Test using precomputed g-values
# l_ij1_pre, d_ij1_pre, u_ij1_pre, R_ij_pre = compute_coefficients_call(w, x_grid, N, K, i, j, dx, dt, B_l, B_u, lambda_n, lambda_p, Y, r, nu, precomputed_g_values)
# print("Pre-computed:")
# print("l_ij1, d_ij1, u_ij1, R_ij =", l_ij1_pre, d_ij1_pre, u_ij1_pre, R_ij_pre)

# # Test using dynamic (computed) g-values
# l_ij1_dyn, d_ij1_dyn, u_ij1_dyn, R_ij_dyn = compute_coefficients_call(w, x_grid, N, K, i, j, dx, dt, B_l, B_u, lambda_n, lambda_p, Y, r, nu, precomputed_g_values=None)
# print("Computed:")
# print("l_ij1, d_ij1, u_ij1, R_ij =", l_ij1_dyn, d_ij1_dyn, u_ij1_dyn, R_ij_dyn)

In [35]:
# Compute boundary conditions
def apply_boundary_conditions(N, h, I, d, u, w):
    """ Adjust for boundary conditions """
    w[0, :] = 0 #(2 / (1 + h/2)) * w[1, :] - (h/2 / (1 + h/2)) * w[2, :]
    w[-1, :] = 0 #(- (1 + h/2) / (1 - h/2)) * w[-2, :] + (2 / (1 - h/2)) * w[-3, :]
    return w

## **Discretized PIDE Equation in Matrix Form**

We discretize the PIDE as:

$$
l_{i,j+1} w_{i-1,j+1} + d_{i,j+1} w_{i,j+1} + u_{i,j+1} w_{i+1,j+1} = w_{i,j} + \frac{\Delta \tau}{\nu} R_{i,j}
$$

where:
- $w_{i,j}$ is the solution at time step $j$ for spatial grid point $i$.
- $l_i, d_i, u_i$ are the tridiagonal coefficients.
- $R_{i,j}$ represents the integral correction term.

### **Matrix Form**
We rewrite the system as:

$$
A \cdot w^{j+1} = \text{rhs}
$$

where:
- **$A$** is an $(N-1) \times (N-1)$ **tridiagonal matrix**:

$$
A =
\begin{bmatrix}
d_1 & u_1 & 0 & 0 & \dots & 0 \\
l_2 & d_2 & u_2 & 0 & \dots & 0 \\
0 & l_3 & d_3 & u_3 & \dots & 0 \\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & l_{N-2} & d_{N-2} & u_{N-2} \\
0 & 0 & 0 & 0 & l_{N-1} & d_{N-1}
\end{bmatrix}
$$

where:
- **$l_i$ (lower diagonal):** Coefficients of $w_{i-1, j+1}$
- **$d_i$ (main diagonal):** Coefficients of $w_{i, j+1}$
- **$u_i$ (upper diagonal):** Coefficients of $w_{i+1, j+1}$

### **Solution Vector**
The unknowns at time $j+1$ form the vector:

$$
w^{j+1} =
\begin{bmatrix}
w_{1,j+1} \\
w_{2,j+1} \\
w_{3,j+1} \\
\vdots \\
w_{N-1,j+1}
\end{bmatrix}
$$

### **Right-Hand Side (RHS)**
The right-hand side includes the known solution at $j$-th time step and the integral correction:

$$
\text{rhs} =
\begin{bmatrix}
w_{1,j} \\
w_{2,j} \\
w_{3,j} \\
\vdots \\
w_{N-1,j}
\end{bmatrix}
+ \frac{\Delta \tau}{\nu}
\begin{bmatrix}
R_{1,j} \\
R_{2,j} \\
R_{3,j} \\
\vdots \\
R_{N-1,j}
\end{bmatrix}
$$

### **Final System to Solve at Each Time Step**
$$
A w^{j+1} = w^j + \frac{\Delta \tau}{\nu} R^j
$$

where:
- **$A$**: $(N-1) \times (N-1)$ **tridiagonal matrix**.
- **$w^{j+1}$**: $(N-1) \times 1$ **vector of unknowns**.
- **$w^j$**: $(N-1) \times 1$ **vector of knowns at previous step**.
- **$R^j$**: $(N-1) \times 1$ **integral term at previous step**.

### **Option Price Computation**

The option price is given by $w^0$, which corresponds to the solution of the PIDE at $\tau = 0$. 

Using the backward recurrence relation:

$$
A w^{j+1} = w^j + \frac{\Delta \tau}{\nu} R^j
$$

we iterate backward in time until $j = 0$, obtaining:

$$
w^0 = A^{-1} \left( w^1 - \frac{\Delta \tau}{\nu} R^0 \right)
$$

To solve for $w^j$ efficiently, we can employ:
- **An iterative approach**, solving $w^j = A^{-1} ( w^{j+1} - \frac{\Delta \tau}{\nu} R^j )$ backward in time.
- **LU factorization**, which decomposes $A = LU$, allowing for efficient forward and backward substitution.
- **The Thomas algorithm**, which provides an $O(N)$ solution for tridiagonal matrices.

### **Extracting the Option Price**
The final option price is given by:

$$
\text{Option Price} = w^0[x_0]
$$

where $x_0 = \log(S_0)$ is the closest grid point to the initial stock price.


In [36]:
def thomas_algorithm(A, b):
    """
    Solves a tridiagonal system A w = b using the Thomas Algorithm (O(N)).

    Parameters:
        A (numpy.ndarray): Tridiagonal matrix of size NxN.
        b (numpy.ndarray): Right-hand side vector of size N.

    Returns:
        numpy.ndarray: Solution vector w.
    """
    N = len(b)
    
    # Extract diagonals from A and create copies
    lower_diag = np.copy(np.diag(A, k=-1))  # l_i (sub-diagonal)
    main_diag = np.copy(np.diag(A, k=0))    # d_i (main diagonal)
    upper_diag = np.copy(np.diag(A, k=1))    # u_i (super-diagonal)

    # Make a copy of b to avoid modifying the input
    b = np.copy(b)

    # Forward elimination
    for i in range(1, N):
        factor = lower_diag[i - 1] / main_diag[i - 1]
        main_diag[i] -= factor * upper_diag[i - 1]
        b[i] -= factor * b[i - 1]

    # Back substitution
    w = np.zeros(N)
    w[-1] = b[-1] / main_diag[-1]
    for i in range(N - 2, -1, -1):
        w[i] = (b[i] - upper_diag[i] * w[i + 1]) / main_diag[i]

    return w

In [37]:
def iterative_solver(A, b, tol=1e-6, max_iter=1000):
    """
    Solves Ax = b using the Gauss-Seidel iterative method.

    Parameters:
        A (numpy.ndarray): Coefficient matrix.
        b (numpy.ndarray): Right-hand side vector.
        tol (float): Convergence tolerance.
        max_iter (int): Maximum number of iterations.

    Returns:
        numpy.ndarray: Solution vector w.
    """
    N = len(b)
    w = np.zeros(N)  # Initial guess (zeros)
    
    for _ in range(max_iter):
        w_old = w.copy()
        
        # Gauss-Seidel iteration
        for i in range(N):
            sum_lower = sum(A[i, j] * w[j] for j in range(i))  # Lower triangular part
            sum_upper = sum(A[i, j] * w_old[j] for j in range(i + 1, N))  # Upper part
            
            w[i] = (b[i] - sum_lower - sum_upper) / A[i, i]
        
        # Check for convergence
        if np.linalg.norm(w - w_old, ord=np.inf) < tol:
            break

    return w

In [38]:
def solve_uoc_pide(x_grid, S0, r, q, Y, K, N, M, dx, dt, nu, lambda_p, lambda_n, method="LU", pre_computed=True, precomputed_g_values=None):
    """
    Solves the UOC option pricing PIDE using an explicit–implicit finite difference scheme.
    
    The PIDE is discretized as:
        l_{i,j+1} * w_{i-1,j+1} + d_{i,j+1} * w_{i,j+1} + u_{i,j+1} * w_{i+1,j+1} 
            = w_{i,j} + (dt/nu) * R_{i,j},
    for interior nodes, while the boundary conditions w(x0, tau)=0 and w(xN, tau)=0 are imposed.
    
    Parameters:
        method (str): "LU", "Thomas", or "Iterative" for solving the linear system.
        
    Returns:
        float: The computed option price at S0 (i.e. at x0 = ln(S0)).
    """
    # Initialize solution matrix w(x, tau) with shape (N, M)
    w = np.zeros((N, M))
    
    # Set initial condition: w(x, 0) = (exp(x) - K)^+ for a call option.
    w[:, 0] = np.maximum(np.exp(x_grid) - K, 0)
    
    # Impose boundary conditions for all time levels (optional since they are re-imposed in the loop)
    w = apply_boundary_conditions(N, dx, 0, 0, 0, w)
    # w[0, :] = 0     # left boundary: x = x_min
    # w[-1, :] = 0    # right boundary: x = x_max (barrier)
    
    # Compute B_l and B_u coefficients
    B_l, B_u = compute_B_coefficients(dx, dt, lambda_n, lambda_p, nu, Y, r, q, pre_computed=True)
    # print(f"B_l: {B_l}, B_u: {B_u}")
    
    # Preallocate matrix A and vector b (of size N, corresponding to the full grid)
    A = np.zeros((N, N))
    b = np.zeros(N)
    
    # Time-marching backward: from j = M-1 down to j = 1.
    for j in range(M - 1, 0, -1):
        # Reset A and b for current time step
        A.fill(0)
        b.fill(0)
        R = np.zeros(N)
        
        # For interior nodes i = 1,..., N-2, compute coefficients and build A and R.
        for i in range(1, N - 1):
            # At point x_i, t_{j+1}
            l_i, d_i, u_i, R[i] = compute_coefficients_call(w, x_grid, N, K, i, j, dx, dt, B_l, B_u, lambda_n, lambda_p, Y, r, nu, precomputed_g_values)
            A[i, i - 1] = l_i
            A[i, i]     = d_i
            A[i, i + 1] = u_i
        
        # Enforce boundary conditions in the linear system:
        # For the left boundary (i=0) and right boundary (i=N-1), we want w=0.
        A[0, :] = 0
        A[0, 0] = 1
        b[0] = 0
        
        A[-1, :] = 0
        A[-1, -1] = 1
        b[-1] = 0
        
        # print("R = ", R)
        
        # Build right-hand side for interior nodes:
        b[1:-1] = w[1:-1, j] + (dt / nu) * R[1:-1]
        
        # Solve the linear system A * sol = b using the chosen method.
        if method == "LU":
            L, U = la.lu(A, permute_l=True)
            sol = la.solve(U, la.solve(L, b))
        elif method == "Thomas":
            sol = thomas_algorithm(A, b)
        elif method == "Iterative":
            sol = iterative_solver(A, b)
        elif method == "scipy":
            sol = la.solve(A, b)
        else:
            raise ValueError("Invalid method. Choose from 'LU', 'Thomas', or 'Iterative'.")
        
        # Update solution at time step j-1 with the computed solution.
        w[:, j - 1] = sol
        
        # Re-impose the boundary conditions explicitly.
        w[0, j - 1] = 0
        w[-1, j - 1] = 0

    # Find the index corresponding to x0 = ln(S0)
    x0_index = np.argmin(np.abs(x_grid - np.log(S0)))
    
    # Return the option price at the node closest to S0
    # print("w =", w)
    return w[x0_index, 0]


In [39]:
# Parameters
S0 = 1900  # Spot price
K = 2000  # Strike price
B = 2200  # Barrier
r = 0.0025  # Risk-free rate 
q = 0.015  # Dividend rate
T = 0.5  # Maturity
sigma = 0.25  # Volatility
nu = 0.31  # Jump intensity
theta = -0.25  # Mean jump size
Y = 0.4  # Tail index

# Grid parameters
N = 100  # Number of spatial steps
M = 100  # Number of time steps

# Define the spatial grid range in log-space
x_min = np.log(S0) - 2 # Lower bound in log-space
x_max = np.log(B)  # Upper bound (log barrier level)

# D = 
# x_i = x_min + i * dx, i = 0, 1, ..., N
# t_j = 0 + j * dt, j = 0, 1, ..., M
dx = (x_max - x_min) / (N-1)  # Spatial step size
dt = T / M  # Time step size

# Discretized spatial and time grids
x_grid = np.linspace(x_min, x_max, N)
tau_grid = np.linspace(0, T, M)  # M+1 grid points in time

# Display the computed grid parameters
grid_params = {
    "x_min": x_min,
    "x_max": x_max,
    "dx": dx,
    "dt": dt,
    "Number of spatial points": N,
    "Number of time points": M,
}

grid_params

{'x_min': 5.549609165154532,
 'x_max': 7.696212639346407,
 'dx': 0.021682863375675505,
 'dt': 0.005,
 'Number of spatial points': 100,
 'Number of time points': 100}

In [40]:
# Compute lambda parameters
lambda_p = ((theta**2 / sigma**4) + (2 / (sigma**2 * nu)))**(1/2) - (theta / sigma**2)
lambda_n = ((theta**2 / sigma**4) + (2 / (sigma**2 * nu)))**(1/2) + (theta / sigma**2)

print("lambda_p =", lambda_p, "lambda_n =", lambda_n)

lambda_p = 14.919057031246467 lambda_n = 6.919057031246467


In [41]:
# Precompute g1 and g2 values for efficiency
precomputed_g_values = {
    "g1_n": np.array([g1(k * lambda_n * dx) for k in range(1, N+1)]),
    "g1_p": np.array([g1(k * lambda_p * dx) for k in range(1, N+1)]),
    "g2_n": np.array([g2(k * lambda_n * dx) for k in range(1, N+1)]),
    "g2_p": np.array([g2(k * lambda_p * dx) for k in range(1, N+1)]),
    "g2_n_shifted": np.array([g2(k * (lambda_n + 1) * dx) for k in range(1, N+1)]),
    "g2_p_shifted": np.array([g2(k * (lambda_p + 1) * dx) for k in range(1, N+1)])
}

In [42]:
# Choose the solver method: "LU", "Thomas", or "Iterative"
solver_method = "scipy"  # Change this to "LU" or "Iterative" if needed

# Compute the option price using the chosen method
option_price = solve_uoc_pide(x_grid, S0, r, q, Y, K, N, M, dx, dt, nu, lambda_p, lambda_n, method=solver_method, pre_computed=True, precomputed_g_values=precomputed_g_values)

# Print the computed option price
print(f"Computed Up-and-Out Call Option Price using {solver_method} method: {option_price:.4f}")

Computed Up-and-Out Call Option Price using scipy method: 1.6268


In [44]:
import numpy as np
import scipy.linalg as la
import scipy.integrate as spi
import itertools
import pandas as pd

# Define grid setup function:
def setup_grid(delta, N, M):
    x_min = np.log(K) - delta
    x_max = np.log(B)
    dx = (x_max - x_min) / (N-1)
    dt = T / (M-1)
    x_grid = np.linspace(x_min, x_max, N)
    tau_grid = np.linspace(0, T, M)
    grid_params = {"x_min": x_min, "x_max": x_max, "dx": dx, "dt": dt, "N": N, "M": M}
    return grid_params, x_grid, dx, dt


# Precompute g-values for given dx and N:
def compute_precomputed_g(dx, N):
    precomputed_g_values = {
        "g1_n": np.array([g1(k * lambda_n * dx) for k in range(1, N)]),
        "g1_p": np.array([g1(k * lambda_p * dx) for k in range(1, N)]),
        "g2_n": np.array([g2(k * lambda_n * dx) for k in range(1, N)]),
        "g2_p": np.array([g2(k * lambda_p * dx) for k in range(1, N)]),
        "g2_n_shifted": np.array([g2(k * (lambda_n + 1) * dx) for k in range(1, N)]),
        "g2_p_shifted": np.array([g2(k * (lambda_p - 1) * dx) for k in range(1, N)])
    }
    return precomputed_g_values


# Sensitivity analysis over delta, N, M:
delta_values = [0.5, 1.0, 1.5, 2.0]  # for x_min = ln(K) - delta
N_values = [50, 100, 200]            # spatial steps
M_values = [50, 100, 200]            # time steps

results = []
for delta, N_val, M_val in itertools.product(delta_values, N_values, M_values):
    grid_params, x_grid, dx, dt = setup_grid(delta, N_val, M_val)
    precomputed_g = compute_precomputed_g(dx, N_val)
    # Use N_val and M_val in the solver call (not global N, M)
    option_price = solve_uoc_pide(x_grid, S0, r, q, Y, K, N_val, M_val, dx, dt, nu, lambda_p, lambda_n, 
                                  method="scipy", pre_computed=True, precomputed_g_values=precomputed_g)
    results.append({
        "delta": delta,
        "N": N_val,
        "M": M_val,
        "OptionPrice": option_price
    })

df = pd.DataFrame(results)
print(df)


    delta    N    M  OptionPrice
0     0.5   50   50   -14.116393
1     0.5   50  100   -16.683608
2     0.5   50  200   -18.452836
3     0.5  100   50    -9.527409
4     0.5  100  100    -9.541758
5     0.5  100  200    -9.846425
6     0.5  200   50     0.618968
7     0.5  200  100     0.618966
8     0.5  200  200     0.618801
9     1.0   50   50  -135.415129
10    1.0   50  100  -140.925286
11    1.0   50  200  -142.600582
12    1.0  100   50     3.770237
13    1.0  100  100   -21.921798
14    1.0  100  200   -45.418507
15    1.0  200   50     1.107604
16    1.0  200  100    -1.879537
17    1.0  200  200    -7.462681
18    1.5   50   50    19.234637
19    1.5   50  100    18.718283
20    1.5   50  200    18.493316
21    1.5  100   50  -167.049930
22    1.5  100  100  -175.458122
23    1.5  100  200  -177.381478
24    1.5  200   50   -11.744591
25    1.5  200  100   -15.151852
26    1.5  200  200   -17.167054
27    2.0   50   50    29.663659
28    2.0   50  100    29.665983
29    2.0 

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.express as px

pivot_table = df.pivot(index='N', columns='M', values='OptionPrice')
N_vals = pivot_table.index.values
M_vals = pivot_table.columns.values
Z = pivot_table.values

# Create an interactive surface plot
fig = go.Figure(data=[go.Surface(z=Z, x=M_vals, y=N_vals, colorscale='Viridis')])
fig.update_layout(title='Option Price Sensitivity (Interactive Surface)',
                  scene=dict(
                      xaxis_title='M (Time steps)',
                      yaxis_title='N (Spatial steps)',
                      zaxis_title='Option Price'),
                  autosize=True)
fig.show()
